In [ ]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [ ]:
import src.pipeline.pre_processing as pre_processing
import src.pipeline.output_schemas as output_schemas
import pandas as pd
import time
import glob
import os

In [ ]:
# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/pre-processing/*")
for f in files:
    os.remove(f)

In [ ]:
start_time = time.time()
current_year = 2022

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [ ]:
cdc = pre_processing.prepare_cdc_data('data/cdc.csv', current_year)

In [ ]:
#cdc.to_csv('output/pre-processing/cdc.csv')
cdc

## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [ ]:
census = pre_processing.prepare_census_data('data/census_workforce.xlsx', 'data/census_pupils.csv')

#todo - add logic for highest / lowest age from census data

In [ ]:
#census.to_csv('output/pre-processing/census.csv')
census

## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [ ]:
sen = pre_processing.prepare_sen_data('data/sen.csv')

In [ ]:
#sen.to_csv("output/pre-processing/sen.csv")
sen

## KS2 and KS4 processing

In [ ]:
#ks2 = pre_processing.prepare_ks2_data('data/ks2.xlsx')
ks2 = pd.DataFrame({'URN':[100,200]})

In [ ]:
#ks2.to_csv('output/pre-processing/ks2.csv')
ks2

In [ ]:
#ks4 = pre_processing.prepare_ks4_data('data/ks4.xlsx')
ks4 = pd.DataFrame({'URN':[100,200]})

In [ ]:
#ks4.to_csv('output/pre-processing/ks4.csv')
ks4

## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [ ]:
academy_ar = pre_processing.prepare_aar_data('data/academy_ar.xlsx')

In [ ]:
#academy_ar.to_csv('output/pre-processing/academy_ar.csv')
academy_ar

Create a summary table for the AR stance of each distinct academy in the table.

Now compute the trust financial position in the same manor as the individual academy position

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [ ]:
schools = pre_processing.prepare_schools_data('data/gias.csv','data/gias_links.csv')


In [ ]:
#schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [ ]:
academies = pre_processing.build_academy_data('data/academy_master_list.csv', 
                                              current_year, schools, census, sen, cdc, 
                                              academy_ar, ks2, ks4)

In [ ]:
#academies.to_csv('output/pre-processing/academies.csv', columns=output_schemas.academies_output)
academies.sort_index()

Merge required census and cdc data to the maintained schools data set

In [ ]:
# Load raw list from CSV
maintained_schools = pre_processing.build_maintained_school_data('data/maintained_schools_master_list.csv',current_year, schools, census, sen, cdc, ks2, ks4)

In [ ]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv', columns=output_schemas.maintained_schools_output)
#maintained_schools

## Federation Capture




In [ ]:
(hard_federations, soft_federations) = pre_processing.build_federations_data('data/gias_all_links.csv', maintained_schools)

In [ ]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

In [ ]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations[['LAEstab']]

In [ ]:
# amalgamate school data

list_of_laestabs = maintained_schools["LAEstab"][maintained_schools["Lead school in federation"]!="0"]
list_of_ukprns = maintained_schools.index[maintained_schools["Lead school in federation"]!="0"]
mappings = dict(zip(list_of_laestabs,list_of_ukprns))

maintained_schools["Federation Lead School UKPRN"] = maintained_schools["Lead school in federation"].map(mappings)
maintained_schools = pd.merge(maintained_schools, hard_federations[['FederationName']], how='left', left_index=True, right_index=True)
maintained_schools.rename(columns = {"FederationName":"Federation Name"}, inplace = True)
maintained_schools = maintained_schools[~maintained_schools.index.duplicated()]

all_school_data = pd.concat((maintained_schools, academies))

all_school_data.to_csv('output/pre-processing/schools.csv', columns=output_schemas.schools_table_output)

# Budget Forcast Returns

In [ ]:
year = 2023

bfr_cell_mapping_cols = {'EFALineNo':'Int64','balance_flag':'Int64'}

bfr_sofa_index_col = [] # TODO define this as necessary
bfr_sofa_cols = {'TrustUPIN':'Int64','CreatedBy':'string','Category':'string','Title':'string','EFALineNo':'Int64','Y1P1':'float','Y1P2':'float','Y2P1':'float','Y2P2':'float'}


bfr_3y_cols = {'TrustUPIN':'Int64','EFALineNo':'Int64','Y2':'Int64','Y3':'Int64','Y4':'Int64'}




def build_bfr_data(bfr_sofa_data_path,bfr_3y_data_path, bfr_master_list_data_path, bfr_cell_mapping_path, academies):

    bfr_cell_mapping = pd.read_csv(
        bfr_cell_mapping_path,
        dtype=bfr_cell_mapping_cols,
        usecols=bfr_cell_mapping_cols.keys(),
)

    bfr_sofa = pd.read_csv(
        bfr_sofa_data_path,
        encoding='unicode-escape',
#        index_col=bfr_sofa_index_col,
        dtype=bfr_sofa_cols,
        usecols=bfr_sofa_cols.keys(),
    )

    bfr_3y = pd.read_csv(
        bfr_3y_data_path,
        encoding='unicode-escape',
#        index_col=bfr_sofa_index_col,
        dtype=bfr_3y_cols,
        usecols=bfr_3y_cols.keys(),
    )    

    bfr_sofa = pd.merge(bfr_sofa, bfr_cell_mapping, how='left', on='EFALineNo')

    bfr_sofa[f'{year}-08-31'] = bfr_sofa['Y2P2']
    bfr_sofa.loc[bfr_sofa['balance_flag']==0, f'{year}-08-31'] = bfr_sofa['Y2P1'] + bfr_sofa['Y2P2']
    bfr_sofa.loc[bfr_sofa['EFALineNo'] == 999, f'{year}-08-31'] = bfr_sofa['Y1P2'][bfr_sofa['EFALineNo'] == 999]

    bfr_sofa.rename(columns={'Y1P1':f'{year}-03-31'}, inplace=True)
    bfr_sofa.drop_duplicates(inplace=True)

    bfr_3y.rename(columns={'Y2':f'{year+1}-08-31','Y3':f'{year+2}-08-31','Y4':f'{year+3}-08-31'}, inplace=True)
    
    bfr_3y.drop_duplicates(inplace=True)


    bfr = pd.merge(bfr_sofa, bfr_3y, how='left', on=('TrustUPIN','EFALineNo'))

    bfr = pd.merge(bfr, academies[['Academy Trust UPIN','NumberOfPupils']].rename(columns={'Academy Trust UPIN':'TrustUPIN'}), how='left',on='TrustUPIN')
    
    
    # vol_stab_flag only valid for end of year finances (e.g. XXXX-08-31) from EFALineNo 0, 430 and 4300    
    # TODO we may need a check for the number of schools who have not submitted a BFR

    return bfr_sofa, bfr_3y, bfr

In [ ]:
bfr_sofa, bfr_3y, bfr = build_bfr_data('data/BFR_SOFA_raw.csv','data/BFR_3Y_raw.csv','data/BFR_master_list_raw.xlsx', 'data/BFR_cell_mapping.csv', academies)

In [ ]:
bfr

### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')

There are 327 duplicates in the academies outputs, and 346 in the maintained (excluding federations)


academy_ar has 10444 entries, 148 of the urns in this list are duplicated, though they look to be schools which have changed from SAT to MAT

The academies_list containes duplicated LAEstabs due to schools transitioning between SAT / MAT etc.

maintained_schools has 10650 entries, 347 of which are nulls. These can just be dropped

There are a few duplicates in federation data as well: